In [1]:
import svox2
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.autograd import Variable

In [2]:
intr = np.array([[380,   0, 320, 0],
                 [  0, 380, 180, 0],
                 [  0,   0,   1, 0],
                 [  0,   0,   0, 0]])

c2w = np.eye(4)

In [3]:
def get_camera_rays(W, H, intrinsics, c2w):
    u, v = np.meshgrid(np.arange(W), np.arange(H))

    u = u.reshape(-1).astype(dtype=np.float32) + 0.5  # add half pixel
    v = v.reshape(-1).astype(dtype=np.float32) + 0.5
    pixels = np.stack((u, v, np.ones_like(u)), axis=0)  # (3, H*W)

    rays_d = np.dot(np.linalg.inv(intrinsics[:3, :3]), pixels) # sensor's location to camera

    rays_d = np.dot(c2w[:3, :3], rays_d)  # (3, H*W)
    rays_d = rays_d.transpose((1, 0))  # (H*W, 3)

    rays_o = c2w[:3, 3].reshape((1, 3))
    rays_o = np.tile(rays_o, (rays_d.shape[0], 1))  # (H*W, 3)

    rays = svox2.Rays(torch.from_numpy(rays_o).float().cuda(), torch.from_numpy(rays_d).float().cuda().contiguous())

    return rays

rays = get_camera_rays(1, 1, intr, c2w)

In [4]:
grid = svox2.SparseGrid(reso=12).requires_grad_(False)
grid.density_data.normal_()
grid.sh_data.normal_()
grid.basis_data.normal_()
grid = grid.cuda().requires_grad_(True)
grid = grid.float()

depth = f(rays)
depth_np = depth.view(640, 480).detach().cpu().numpy()
plt.imshow(depth_np)

In [5]:
rays_data = svox2.Rays(rays.origins, rays.dirs)
rays_data.origins = rays_data.origins.cuda().requires_grad_(True)
rays_data.dirs = rays_data.dirs.cuda().requires_grad_(True)
# rays_data = rays_data.float()

In [6]:
# density_data = grid.density_data
# inp = (torch.rand_like(density_data)).float().requires_grad_(True)

# rays_o_data = rays.origins
# inp = rays_o_data.float().requires_grad_(True)

rays_o_data = rays_data.origins
inp = (torch.rand_like(rays_o_data)).float().requires_grad_(True)

In [7]:
def fr(x):
    cgrid = grid._to_cpp()
    cgrid.density_data = cgrid.density_data.float()
    cgrid.sh_data = cgrid.sh_data.float()
    
#     rays_data = svox2.Rays(x, rays.dirs)
    crays_data = rays_data._to_cpp()
    crays_data.origins = x
    crays_data.dirs = crays_data.dirs.float()
    out = svox2.svox2._RayRenderFunction.apply(grid.density_data, grid.sh_data, None, None,
                                                  cgrid, crays_data, grid.opt._to_cpp())
    #print(out.mean(), out.std())
#     print('This is', x.mean())
#     return torch.ones_like(out) * x.view(-1,)[0]
    
    return out

def fv(x):
    cgrid = grid._to_cpp()
    cgrid.density_data = x
    cgrid.background_data = False
    cgrid.basis_data = cgrid.basis_data.double()
    cgrid.background_data = cgrid.background_data.double()
    cgrid.sh_data = cgrid.sh_data.double()
    out = svox2.svox2._VolumeRenderFunction.apply(x, grid.sh_data, None, None,
                                                  cgrid, rays._to_cpp(), grid.opt._to_cpp(), "cuvol")
    #print(out.mean(), out.std())
    return out

def fd(x):
    cgrid = grid.float()._to_cpp()
    cgrid.density_data = x
    out = svox2.svox2._DepthRenderFunction.apply(x, 
                                                  cgrid, rays._to_cpp(), grid.opt._to_cpp())
    #print(out.mean(), out.std())
    return out

In [8]:
@torch.no_grad()
def finite_diff(f, x, eps=1e-4):
    N = x.shape[-1]
    g = torch.zeros(N, 3)
    I = torch.eye(N).type_as(x)
    for i in range(N):
        f1 = f(x + eps * I[i])
        f2 = f(x - eps * I[i])
        g[i] = (f1 - f2) / (2.0 * eps)
    return g

def autodiff(f, x):
#     x = x.requires_grad_(True)
#     y = f(x)
#     g = torch.autograd.grad(y, x)[0]
    N = x.shape[-1]
    g = torch.zeros(N, 3)
    x = x.requires_grad_(True)
    y = f(x)
    print(f"{y.shape=:}, {x.shape=:}, {g.shape=:}")

    for i in range(3):
        y[0][i].backward()
        x.grad
        
#         print(torch.autograd.grad(y[0][i], x, retain_graph=True, allow_unused=False))
#         g[:, i] = (torch.autograd.grad(y[0][i], x, retain_graph=True, allow_unused=False)[0]).reshape(-1)
    return g

def relative_diff(f1, f2, eps=1e-12):
    f1n = f1.norm()
    f2n = f2.norm()
    if f2n < eps:
        return torch.tensor(0.)
    
    fdn = (f2 - f1).norm()
    return fdn / f2n

In [9]:
# fdiff = finite_diff(fr, inp, eps=1e-1)
def dummy_fr(x):
    return x + 1.
fad = autodiff(fr, inp)
# fad = autodiff(dummy_fr, inp)

y.shape=torch.Size([1, 3]), x.shape=torch.Size([1, 3]), g.shape=torch.Size([3, 3])


RuntimeError: function _RayRenderFunctionBackward returned a gradient different than None at position 6, but the corresponding forward input was not a Variable

In [ ]:
relative_diff(fad, fdiff)

In [ ]:
eps = np.linspace(0.1, 1e-17, 1000)
err = []
rerr = []
fdn = []
fan = []
for ep in eps:
#     f1 = autodiff(fr, inp)
    fdiff = finite_diff(fr, inp, eps=ep)
#     err.append((f1 - fdiff).norm().item())
#     rerr.append(relative_diff(f1, fdiff).item())
    fdn.append(fdiff.norm().item())
#     fan.append(f1.norm().item())

In [ ]:
xaxis = -1 * np.log10(eps)
plt.style.use("seaborn-whitegrid")
# plt.plot(xaxis, err, label="Absolute Error", c ='orange')
# plt.plot(xaxis, rerr, label="Relative Error")
plt.plot(xaxis, fdn, label="Norm of finite difference gradient")
# plt.plot(xaxis, fan, label="Norm of analytical gradient")
plt.legend()
# plt.xlabel("-Log10(Epsilon)")
# plt.ylabel("Error")
# plt.title("Relative gradient error as a function of epsilon")